In [ ]:
import pandas as pd
import geopandas as gpd

In [ ]:
CURT=gpd.read_file(r"D:\Secretaría\Bases_shp_INEGI\CURT\024 Atlacomulco\ATLACOMULCO_PREDIOS.shp")
CURT

In [ ]:
casas_filtro=gpd.read_file(r"D:\Secretaría\cruces_bases\limpiezas_shp_base\Atlacomulco\Atlacomulco_casas_limpio.shp")
# casas_filtro.plot()

In [ ]:
# CURT.geometry.value_counts()
casas_filtro.geometry.value_counts()

In [ ]:
curtxcasas = gpd.read_file(r"D:\Secretaría\cruces_bases\limpiezas_shp_base\Atlacomulco\Curtxdetect\Cruce_casasxcurt.shp")
curtxcasas

In [ ]:
#Se revisa el CRS de cada base para poder hacer el cruce entre ellas
print("crs CURT: ",CURT.crs)
print("crs casas: ",casas_filtro.crs)

In [ ]:
CURT=CURT.set_crs(6364,allow_override=True)
# CURT.geometry.head()
CURT=CURT.to_crs(3857)


In [ ]:
CURT.shape, casas_filtro.shape

In [ ]:
cruce_curt=gpd.sjoin(CURT,casas_filtro)

In [ ]:
cruce_curt['cruzaron']=1
cruce_curt

In [ ]:
cruce_curt.geometry.unique()

In [ ]:
no_cruzanxcurt=CURT[~(CURT.geometry.isin(cruce_curt['geometry']))]
no_cruzanxcurt

In [ ]:
no_cruzanxcurt['cruzaron']=0
no_cruzanxcurt

In [ ]:
cruce_curt.shape, no_cruzanxcurt.shape

In [ ]:
cruce_final_curt=pd.concat([cruce_curt,no_cruzanxcurt])

In [ ]:
#Para detectar duplicados x geometría
geo_dupli=cruce_final_curt.geometry.value_counts()[cruce_final_curt.geometry.value_counts()>1].index
cruce_final_curt.loc[cruce_final_curt.geometry.isin(geo_dupli),"dupGEO"]=1
cruce_final_curt.loc[cruce_final_curt.dupGEO != 1, "dupGEO"]=0
cruce_final_curt


In [ ]:
#Para detectar duplicados x curt
curt_dupli=cruce_final_curt.curt.value_counts()[cruce_final_curt.curt.value_counts()>1].index
cruce_final_curt.loc[cruce_final_curt.curt.isin(curt_dupli),"dupCURT"]=1
cruce_final_curt.loc[cruce_final_curt.dupCURT != 1, "dupCURT"]=0
cruce_final_curt

In [ ]:
cruce_final_curt.to_file(r"D:\Secretaría\cruces_bases\limpiezas_shp_base\Atlacomulco\Curtxdetect/cruce_curtxcasas_Atlacomulco.shp")